# Read COAWST model data from NetCDF

In [ ]:
import xarray as xr
import glob

from dask.distributed import Client
from dask_jobqueue import SLURMCluster

In [ ]:
#cluster = SLURMCluster(processes=1, cores=1, memory='8GB', 
#                       project='woodshole', walltime='00:30:00', queue='normal',
#                      interface='ib0', reservation='acc')

In [ ]:
cluster = SLURMCluster(processes=1, cores=1, memory='8GB', interface='ipogif0',
                       project='woodshole', walltime='00:30:00')

In [ ]:
client = Client(cluster)
print(cluster.job_script())

In [ ]:
cluster.scale(20)

In [ ]:
files = '/caldera/projects/usgs/hazards/cmgp/woodshole/rsignell/EarthMap/Projects/de_bay/run_001/ocean_deb_his_run07A_zo015_*.nc'
ds0 = xr.open_mfdataset(files, parallel=True, chunks={'ocean_time':5})

In [ ]:
ds0 = ds0.chunk({'ocean_time':5})

In [ ]:
zarr_ds = '/caldera/projects/usgs/hazards/cmgp/woodshole/rsignell/EarthMap/Projects/de_bay/zarr_002'


In [ ]:
task = ds0.to_zarr(zarr_ds, consolidated=True, compute=False, mode='w')

In [ ]:
futures = client.compute(task, retries=5)

In [ ]:
client.cancel(futures)

In [ ]:
flist = glob.glob('/caldera/projects/usgs/hazards/cmgp/woodshole/rsignell/EarthMap/Projects/de_bay/run_001/ocean_deb_his_run07A_zo015_*.nc')

In [ ]:
flist.sort()

In [ ]:
flist

In [ ]:
investigate dask retry

In [ ]:
zarr_ds = '/caldera/projects/usgs/hazards/cmgp/woodshole/rsignell/EarthMap/Projects/de_bay/zarr_001'
for file in flist[8:]:
    print(file)
    ds = xr.open_dataset(file, chunks={'ocean_time':1})
    if '00001' in file:
        ds.to_zarr(zarr_ds, consolidated=True, mode='w')
    else:
        ds.to_zarr(zarr_ds, consolidated=True, append_dim='ocean_time')

In [ ]:
5761/721

In [ ]:
cluster.close()